In [162]:
import torch
import torch.nn as nn
import string
import random
import sys
import unidecode
from datetime import datetime
import pickle
# from torch.utils.tensorboard import SummaryWriter


In [163]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [164]:
# get words
all_words = []


with open('./data/unique_words_2k_articles.dat', 'rb') as file:
    all_words = pickle.load(file)

all_words.append(' ')
vocab_length = len(all_words)
print(f'number of words: {vocab_length}')


number of words: 20449


In [165]:
# read file
corpus = []
file_content = []
with open('./data/tokenized_articles.dat', 'rb') as file:
    file_content = pickle.load(file)

# this approach uses embedding, and therefore doesn't need padding so we remove it from the prepared data
print(len(file_content))
for article in file_content:
    corpus.extend([word.strip() for word in article if word not in ['<pad>', ' ']])
# got a value error, so addin a blank space to corpus
# might be a flaw in the data preparation

print(len(corpus))
print(corpus[-30:])


2000
722324
['<period>', 'both', 'mexico', 'and', 'canada', 'have', 'dismissed', 'his', 'musing', 'in', 'a', 'tuesday', 'speech', 'that', '<quotation_mark>', 'well', 'end', 'up', 'probably', 'terminating', 'nafta', 'at', 'some', 'point', '<quotation_mark>', 'as', 'a', 'negotiating', 'tactic', '<period>']


In [166]:
# module

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(self.hidden_size * num_layers, output_size)

    def forward(self, x, hidden, cell):
        out = self.embed(x)
        out, (hidden, cell) = self.lstm(out.unsqueeze(1), (hidden, cell))
        out = self.fc(out.reshape(out.shape[0], -1))
        return out, (hidden, cell)
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers * 2, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers * 2, batch_size, self.hidden_size).to(device)
        return hidden, cell



In [167]:
class Generator():
    def __init__(self):
        self.chunk_len = 30
        self.num_epochs = 10000
        self.batch_size = 1
        self.print_every = self.num_epochs // 20 or 1
        self.hidden_size = 256
        self.num_layers = 2
        self.lr = 0.003


    def word_tensor(self, string):
        tensor = torch.zeros(len(string)).long()
        for c in range(len(string)):
            tensor[c] = all_words.index(string[c])
        return tensor


    def get_random_batch(self):
        start_idx = random.randint(0, len(corpus) - self.chunk_len)
        end_idx = start_idx + self.chunk_len + 1
        text_str = corpus[start_idx:end_idx]
        text_input = torch.zeros(self.batch_size, self.chunk_len)
        text_target = torch.zeros(self.batch_size, self.chunk_len)
        for i in range(self.batch_size):
            text_input[i,:] = self.word_tensor(text_str[:-1])
            text_target[i,:] = self.word_tensor(text_str[1:])
        return text_input.long(), text_target.long()


    def generate(self, initial_str='i would like', predict_len=200, temperature=0.85):
        initial_words = initial_str.split(' ')
        print(initial_words)
        hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)
        initial_input = self.word_tensor(initial_words)
        predicted = initial_words
        
        for p in range(len(initial_words) - 1):
            _, (hidden, cell) = self.rnn(initial_input[p].view(1).to(device), hidden, cell)

        last_word = initial_input[-1]
        for p in range(predict_len):
            output, (hidden, cell) = self.rnn(last_word.view(1).to(device), hidden, cell)
            output_dist = output.data.view(-1).div(temperature).exp()
            top_word = torch.multinomial(output_dist, 1)[0]
            predicted_word = [all_words[top_word]]
            predicted.extend(predicted_word)
            last_word = self.word_tensor(predicted_word)
            


        return predicted


    def train(self):
        self.rnn = RNN(vocab_length, self.hidden_size, self.num_layers, vocab_length).to(device)
        optimizer = torch.optim.Adam(self.rnn.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()
        print(f'<{datetime.now()}>starting training')

        for epoch in range(1, self.num_epochs + 1):
            input, target = self.get_random_batch()
            hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)

            self.rnn.zero_grad()
            loss = 0
            input = input.to(device)
            target = target.to(device)

            for c in range(self.chunk_len):
                output, (hidden, cell) = self.rnn(input[:, c], hidden, cell)
                loss += criterion(output, target[:, c])

            loss.backward()
            optimizer.step()
            loss = loss.item() / self.chunk_len

            if epoch % self.print_every == 0:
                print(f'\n\n<{datetime.now()}> | epoch: {epoch}/{self.num_epochs} | loss: {loss}')
                print(self.generate())


In [168]:
gentext = Generator()
gentext.train()


<2023-05-29 13:32:37.734677>starting training


<2023-05-29 13:33:20.028073> | epoch: 400/10000 | loss: 6.112348937988282
['i', 'would', 'like']
['i', 'would', 'like', 'administration', 'of', 'a', 'susan', 'had', 'now', 'national', 'of', 'a', 'democrats', 'fears', 'and', 'minimizing', 'we', 'been', '<quotation_mark>', 'vetoes', 'to', 'attorney', 'and', 'iran', '<right_paren>', '<comma>', 'already', 'today', '<period>', 'u', '<period>', 's', '<period>', 'has', 'has', 'threatened', 'now', 'to', 'breakthrough', '<comma>', 'virginia', 'steele', 'themselves', '<period>', 's', '<period>', '<quotation_mark>', 'allies', 'would', '<question_mark>', 'gets', 'that', 'our', '<quotation_mark>', 'security', 'right', 'that', 'is', 'it', 'think', 'him', 'and', 'for', 'the', 'march', 'have', 'and', 'this', 'appointee', 'asked', 'british', 'and', 'every', 'would', 'and', 'apo', 'nations', '<comma>', 'as', 'the', 'ending', 'in', 'said', '<comma>', 'adding', '<period>', 'he', 'asked', 'laos', 'to', 'econo

In [174]:
' '.join(gentext.generate(initial_str='i would like', predict_len=400, temperature=0.5)).replace('<quotation_mark>', '"').replace(' <comma>', ',').replace(' <period>', '.')


['i', 'would', 'like']


'i would like a " commission that the u. s. supreme court in the u. s. president donald trump has been more than a strengthening for the constitutional way that, " he said. " it was ". " the official said, " he said. " the house of representatives, a senior u. s. dollar agencies, when the u. s. president donald trump was not been to a former president barack obamas policy in the house of representatives for the u. s. companies, who do not be at the united states, " he said. trump has said he was not " been a request for comment. the pentagon is in a " sanctuary " for the senate on saturday, a democrat, a former campaign manager paul paul ryan, said the plan, the bill would be more, " the official said. " the u. s. debt, who sued said it was a central at the country, said on tuesday that he was a letter to the tax bill that would be a day. " but the u. s. supreme court. " the u. s. president donald trump on saturday. " the administration said, the top democrat on the u. s. election. " t

In [170]:
# m = nn.Linear(512, 100)
# input = torch.randn(1, 512)
# output = m(input)
# print(output)
